# Völuspá analysis

The aim of this notebook is to see how the use of NLP techniques may help philological studies. My first analysis is on Völuspá, the first poem of the Elder Edda, telling the destiny of the worlds.


#### Set your own user path

In [3]:
USER_PATH = "/home/clement"

### Download Old Norse Corpora to cltk_data directory

In [2]:
from cltk.corpus.utils.importer import CorpusImporter
onc = CorpusImporter("old_norse")
onc.import_corpus("old_norse_texts_heimskringla")

Configure **ipython**.

```bash
$ ipython profile create
$ ipython locate
$ nano .ipython/profile_default/ipython_config.py
```
 Add it a the end of the file:
```bash
c.InteractiveShellApp.exec_lines = [
    'import sys; sys.path.append("/home/pi/cltk_data")'
]
```
It is necessary to do that because it makes things easier to utilize data furnished by CLTK. You will see later in the notebook how it is used.

And... It's done!

Install the **kernel** associated with **python3.6** [https://ipython.readthedocs.io/en/stable/install/kernel_install.html](https://ipython.readthedocs.io/en/stable/install/kernel_install.html) 

In [9]:
# old_norse_text_perseus

import os
module_path = os.path.join(USER_PATH, "cltk_data/old_norse/text/old_norse_texts_heimskringla/")
here = os.getcwd()
os.chdir(module_path)
corpus_path = os.path.join(module_path, "Völuspá")
import text_manager
loader = text_manager.TextLoader(os.path.join("Sæmundar-Edda", "Völuspá"), "txt")
text = loader.load()
os.chdir(here)
print(text[:200])


1.
Hljóðs bið ek allar
helgar kindir,
meiri ok minni
mögu Heimdallar;
viltu at ek, Valföðr,
vel fyr telja
forn spjöll fira,
þau er fremst of man.
 
2.
Ek man jötna
ár of borna,
þá er forðum mik
fædda 


#### split text in paragraphs ans verses


In [72]:
import re
def remove_punctuations(text):
    res = text
    # for punctuation in "-:?":
    #    res = "".join(res.split(punctuation))
    res = re.sub("[\-:\?]", "", res)
    res = re.sub("z", "s", res)
    res = re.sub("x", "ks", res)
    res = re.sub(r" +", " ", res)
    return res

In [73]:
import re

indices = [(m.start(0), m.end(0)) for m in re.finditer(r"[0-9]{1,2}\.", text)]  #re.findall(r"[0-9]{1,2}\.", text)
paragraphs = [text[indices[i][1]:indices[i+1][0]].split("\n") for i in range(len(indices)-1)]
paragraphs = [[remove_punctuations(verse).strip() for verse in paragraph if remove_punctuations(verse) != "" and verse != "\xa0"] for paragraph in paragraphs ]
paragraphs

[['Hljóðs bið ek allar',
  'helgar kindir,',
  'meiri ok minni',
  'mögu Heimdallar;',
  'viltu at ek, Valföðr,',
  'vel fyr telja',
  'forn spjöll fira,',
  'þau er fremst of man.'],
 ['Ek man jötna',
  'ár of borna,',
  'þá er forðum mik',
  'fædda höfðu;',
  'níu man ek heima,',
  'níu íviðjur,',
  'mjötvið mæran',
  'fyr mold neðan.'],
 ['Ár var alda,',
  'þat er ekki var,',
  'vara sandr né sær',
  'né svalar unnir;',
  'jörð fannsk æva',
  'né upphiminn,',
  'gap var ginnunga',
  'en gras hvergi.'],
 ['Áðr Burs synir',
  'bjöðum of yppðu,',
  'þeir er Miðgarð',
  'mæran skópu;',
  'sól skein sunnan',
  'á salar steina,',
  'þá var grund gróin',
  'grænum lauki.'],
 ['Sól varp sunnan,',
  'sinni mána,',
  'hendi inni hægri',
  'um himinjöður;',
  'sól þat né vissi,',
  'hvar hon sali átti,',
  'máni þat né vissi,',
  'hvat hann megins átti,',
  'stjörnur þat né vissu',
  'hvar þær staði áttu.'],
 ['Þá gengu regin öll',
  'á rökstóla,',
  'ginnheilög goð,',
  'ok um þat gættusk;',


#### Use the phonology module to transcribe phonetically the text

In [74]:
from cltk.phonology import utils as phu
from cltk.phonology.old_norse import transcription as ont 
tr = phu.Transcriber(ont.DIPHTHONGS_IPA, ont.DIPHTHONGS_IPA_class, ont.IPA_class, ont.old_norse_rules)
for paragraph in paragraphs:
    for verse in paragraph:
        print(repr(verse))
        print(tr.main(verse))

'Hljóðs bið ek allar'
[hljoːðs bið ɛk alːar]
'helgar kindir,'
[hɛlɣar kindir]
'meiri ok minni'
[mɛiri ɔk minːi]
'mögu Heimdallar;'
[mœɣu hɛimdalːar]
'viltu at ek, Valföðr,'
[viltu at ɛk valvœðr]
'vel fyr telja'
[vɛl fyr tɛlja]
'forn spjöll fira,'
[fɔrn spjœlː fira]
'þau er fremst of man.'
[θɒu ɛr frɛmst ɔv man]
'Ek man jötna'
[ɛk man jœtna]
'ár of borna,'
[aːr ɔv bɔrna]
'þá er forðum mik'
[θaː ɛr fɔrðum mik]
'fædda höfðu;'
[fɛːdːa hœvðu]
'níu man ek heima,'
[niːu man ɛk hɛima]
'níu íviðjur,'
[niːu iːviðjur]
'mjötvið mæran'
[mjœtvið mɛːran]
'fyr mold neðan.'
[fyr mɔld nɛðan]
'Ár var alda,'
[aːr var alda]
'þat er ekki var,'
[θat ɛr ɛkːi var]
'vara sandr né sær'
[vara sandr neː sɛːr]
'né svalar unnir;'
[neː svalar unːir]
'jörð fannsk æva'
[jœrð fanːsk ɛːva]
'né upphiminn,'
[neː upːhiminː]
'gap var ginnunga'
[gap var ginːunɣa]
'en gras hvergi.'
[ɛn gras hvɛrɣi]
'Áðr Burs synir'
[aːðr burs synir]
'bjöðum of yppðu,'
[bjœðum ɔv ypːðu]
'þeir er Miðgarð'
[θɛir ɛr miðɣarð]
'mæran skópu;'
[mɛːran

[nuː ɛru talðar]
'nönnur Herjans,'
[nœnːur hɛrjans]
'görvar at ríða'
[gœrvar at riːða]
'grund valkyrjur.'
[grund valkyrjur]
'Ek sá Baldri,'
[ɛk saː baldri]
'blóðgum tívur,'
[bloːðɣum tiːvur]
'Óðins barni,'
[oːðins barni]
'örlög folgin;'
[œrlœɣ fɔlɣin]
'stóð of vaksinn'
[stoːð ɔv vaksinː]
'völlum hæri'
[vœlːum hɛːri]
'mjór ok mjök fagr'
[mjoːr ɔk mjœk faɣr]
'mistilteinn.'
[mistiltɛinː]
'Varð af þeim meiði,'
[varð av θɛim mɛiði]
'er mær sýndisk,'
[ɛr mɛːr syːndisk]
'harmflaug hættlig,'
[harmvlɒuɣ hɛːtːliɣ]
'Höðr nam skjóta;'
[hœðr nam skjoːta]
'Baldrs bróðir var'
[baldrs broːðir var]
'of borinn snemma,'
[ɔv bɔrinː snɛmːa]
'sá nam Óðins sonr'
[saː nam oːðins sɔnr]
'einnættr vega.'
[ɛinːɛːtːr vɛɣa]
'Þó hann æva hendr'
[θoː hanː ɛːva hɛndr]
'né höfuð kembði,'
[neː hœvuð kɛmbði]
'áðr á bál of bar'
[aːðr aː baːl ɔv bar]
'Baldrs andskota;'
[baldrs andskɔta]
'en Frigg of grét'
[ɛn frigː ɔv greːt]
'í Fensölum'
[iː fɛnsœlum]
'vá Valhallar.'
[vaː valhalːar]
'Vituð ér enn eða hvat'
[vituð eːr ɛnː ɛ

[fɛstr mun slitna]
'en freki renna;'
[ɛn frɛki rɛnːa]
'fjölð veit ek fræða'
[fjœlð vɛit ɛk frɛːða]
'fram sé ek lengra'
[fram seː ɛk lɛnɣra]
'um ragna rök'
[um raɣna rœk]
'römm sigtíva.'
[rœmː siɣtiːva]
'Sér hon upp koma'
[seːr hɔn upː kɔma]
'öðru sinni'
[œðru sinːi]
'jörð ór ægi'
[jœrð oːr ɛːɣi]
'iðjagræna;'
[iðjaɣrɛːna]
'falla forsar,'
[falːa fɔrsar]
'flýgr örn yfir,'
[flyːɣr œrn yvir]
'sá er á fjalli'
[saː ɛr aː fjalːi]
'fiska veiðir.'
[fiska vɛiðir]
'Finnask æsir'
[finːask ɛːsir]
'á Iðavelli'
[aː iðavɛlːi]
'ok um moldþinur'
[ɔk um mɔldθinur]
'máttkan dæma'
[maːtːkan dɛːma]
'ok minnask þar'
[ɔk minːask θar]
'á megindóma'
[aː mɛɣindoːma]
'ok á Fimbultýs'
[ɔk aː fimbultyːs]
'fornar rúnir.'
[fɔrnar ruːnir]
'Þar munu eftir'
[θar munu ɛftir]
'undrsamligar'
[undrsamliɣar]
'gullnar töflur'
[gulːnar tœvlur]
'í grasi finnask,'
[iː grasi finːask]
'þærs í árdaga'
[θɛːrs iː aːrdaɣa]
'áttar höfðu.'
[aːtːar hœvðu]
'Munu ósánir'
[munu oːsaːnir]
'akrar vaksa,'
[akrar vaksa]
'böls mun alls batna,'
[b

#### show alliterations in the text

In [81]:
import re
from collections import Counter

for paragraph in paragraphs:
    for verse in paragraph:
        ipa_verse = tr.main(verse)
        print(Counter(re.sub(" ", "", ipa_verse)))
        print(Counter([ipa_verse[i:i+2] for i in range(1,len(ipa_verse)-1) if " " not in ipa_verse[i:i+2]]))

Counter({'l': 2, 'ː': 2, 'ð': 2, 'a': 2, '[': 1, 'h': 1, 'j': 1, 'o': 1, 's': 1, 'b': 1, 'i': 1, 'ɛ': 1, 'k': 1, 'r': 1, ']': 1})
Counter({'hl': 1, 'lj': 1, 'jo': 1, 'oː': 1, 'ːð': 1, 'ðs': 1, 'bi': 1, 'ið': 1, 'ɛk': 1, 'al': 1, 'lː': 1, 'ːa': 1, 'ar': 1, 'r]': 1})
Counter({'r': 2, 'i': 2, '[': 1, 'h': 1, 'ɛ': 1, 'l': 1, 'ɣ': 1, 'a': 1, 'k': 1, 'n': 1, 'd': 1, ']': 1})
Counter({'hɛ': 1, 'ɛl': 1, 'lɣ': 1, 'ɣa': 1, 'ar': 1, 'ki': 1, 'in': 1, 'nd': 1, 'di': 1, 'ir': 1, 'r]': 1})
Counter({'i': 4, 'm': 2, '[': 1, 'ɛ': 1, 'r': 1, 'ɔ': 1, 'k': 1, 'n': 1, 'ː': 1, ']': 1})
Counter({'mɛ': 1, 'ɛi': 1, 'ir': 1, 'ri': 1, 'ɔk': 1, 'mi': 1, 'in': 1, 'nː': 1, 'ːi': 1, 'i]': 1})
Counter({'m': 2, 'a': 2, '[': 1, 'œ': 1, 'ɣ': 1, 'u': 1, 'h': 1, 'ɛ': 1, 'i': 1, 'd': 1, 'l': 1, 'ː': 1, 'r': 1, ']': 1})
Counter({'mœ': 1, 'œɣ': 1, 'ɣu': 1, 'hɛ': 1, 'ɛi': 1, 'im': 1, 'md': 1, 'da': 1, 'al': 1, 'lː': 1, 'ːa': 1, 'ar': 1, 'r]': 1})
Counter({'v': 3, 'l': 2, 't': 2, 'a': 2, '[': 1, 'i': 1, 'u': 1, 'ɛ': 1, 'k': 1,

Counter({'aː': 1, 'rœ': 1, 'œk': 1, 'ks': 1, 'st': 1, 'to': 1, 'oː': 1, 'ːl': 1, 'la': 1, 'a]': 1})
Counter({'g': 2, 'i': 2, '[': 1, 'n': 1, 'ː': 1, 'h': 1, 'ɛ': 1, 'l': 1, 'œ': 1, 'ɣ': 1, 'ɔ': 1, 'ð': 1, ']': 1})
Counter({'gi': 1, 'in': 1, 'nː': 1, 'ːh': 1, 'hɛ': 1, 'ɛi': 1, 'il': 1, 'lœ': 1, 'œɣ': 1, 'gɔ': 1, 'ɔð': 1, 'ð]': 1})
Counter({'k': 2, 'u': 2, 't': 2, 'ː': 2, '[': 1, 'ɔ': 1, 'm': 1, 'θ': 1, 'a': 1, 'g': 1, 'ɛ': 1, 's': 1, ']': 1})
Counter({'ɔk': 1, 'um': 1, 'θa': 1, 'at': 1, 'gɛ': 1, 'ɛː': 1, 'ːt': 1, 'tː': 1, 'ːu': 1, 'us': 1, 'sk': 1, 'k]': 1})
Counter({'n': 2, 'ː': 2, '[': 1, 'o': 1, 't': 1, 'ɔ': 1, 'k': 1, 'i': 1, 'ð': 1, 'j': 1, 'u': 1, 'm': 1, ']': 1})
Counter({'no': 1, 'oː': 1, 'ːt': 1, 'tː': 1, 'ɔk': 1, 'ni': 1, 'ið': 1, 'ðj': 1, 'ju': 1, 'um': 1, 'm]': 1})
Counter({'v': 3, 'n': 2, '[': 1, 'œ': 1, 'ɔ': 1, 'g': 1, 'a': 1, 'ː': 1, 'u': 1, ']': 1})
Counter({'nœ': 1, 'œv': 1, 'vn': 1, 'ɔv': 1, 'ga': 1, 'aː': 1, 'ːv': 1, 'vu': 1, 'u]': 1})
Counter({'[': 1, 'm': 1, 'ɔ': 1,

Counter({'a': 2, 'ɛ': 2, 'r': 2, '[': 1, 'm': 1, 'ː': 1, 'l': 1, 'd': 1, 'v': 1, 'ɣ': 1, ']': 1})
Counter({'ɛr': 2, 'ma': 1, 'aː': 1, 'ːl': 1, 'dv': 1, 'vɛ': 1, 'rɣ': 1, 'ɣa': 1, 'a]': 1})
Counter({'i': 4, 'l': 2, '[': 1, 'ː': 1, 'd': 1, 'v': 1, 'a': 1, 'n': 1, 's': 1, 'ð': 1, ']': 1})
Counter({'li': 2, 'iː': 1, 'dv': 1, 'va': 1, 'al': 1, 'in': 1, 'ns': 1, 'ið': 1, 'ði': 1, 'i]': 1})
Counter({'n': 2, '[': 1, 'l': 1, 'j': 1, 'o': 1, 'ː': 1, 'a': 1, 'k': 1, 'i': 1, 'd': 1, 'u': 1, 'm': 1, ']': 1})
Counter({'lj': 1, 'jo': 1, 'oː': 1, 'ːn': 1, 'na': 1, 'ki': 1, 'in': 1, 'nd': 1, 'du': 1, 'um': 1, 'm]': 1})
Counter({'l': 3, 't': 2, 'a': 2, '[': 1, 'i': 1, 'ɔ': 1, 'v': 1, 'r': 1, 's': 1, 'ɛ': 1, 'j': 1, ']': 1})
Counter({'ti': 1, 'il': 1, 'lɔ': 1, 'ɔv': 1, 'va': 1, 'ar': 1, 'rs': 1, 'tɛ': 1, 'ɛl': 1, 'lj': 1, 'ja': 1, 'a]': 1})
Counter({'ɛ': 2, 'r': 2, 'ː': 2, '[': 1, 'θ': 1, 'i': 1, 's': 1, 'o': 1, 't': 1, 'u': 1, ']': 1})
Counter({'θɛ': 1, 'ɛi': 1, 'ir': 1, 'ɛr': 1, 'so': 1, 'oː': 1, 'ːt':

Counter({'ɛ': 2, 'u': 2, '[': 1, 'r': 1, 'g': 1, 'l': 1, 'ː': 1, 'v': 1, 'i': 1, 'ɣ': 1, ']': 1})
Counter({'ɛr': 1, 'gu': 1, 'ul': 1, 'lː': 1, 'ːv': 1, 'vɛ': 1, 'ɛi': 1, 'iɣ': 1, 'ɣu': 1, 'u]': 1})
Counter({'u': 3, '[': 1, 'g': 1, 'ɛ': 1, 'i': 1, 'r': 1, 'm': 1, 's': 1, 't': 1, 'd': 1, 'ː': 1, ']': 1})
Counter({'gɛ': 1, 'ɛi': 1, 'ir': 1, 'ru': 1, 'um': 1, 'st': 1, 'tu': 1, 'ud': 1, 'dː': 1, 'ːu': 1, 'u]': 1})
Counter({'ː': 3, 'h': 2, '[': 1, 'ɔ': 1, 'k': 1, 'i': 1, 'œ': 1, 'l': 1, 'a': 1, 'r': 1, 's': 1, ']': 1})
Counter({'ɔk': 1, 'iː': 1, 'hœ': 1, 'œl': 1, 'lː': 1, 'ha': 1, 'aː': 1, 'ːr': 1, 'rs': 1, 's]': 1})
Counter({'a': 2, 'n': 2, '[': 1, 'h': 1, 'b': 1, 'r': 1, 'ɛ': 1, 'ː': 1, 'd': 1, 'u': 1, ']': 1})
Counter({'ha': 1, 'an': 1, 'na': 1, 'br': 1, 'rɛ': 1, 'ɛn': 1, 'nː': 1, 'ːd': 1, 'du': 1, 'u]': 1})
Counter({'r': 3, '[': 1, 'θ': 1, 'i': 1, 's': 1, 'v': 1, 'a': 1, 'b': 1, 'ɛ': 1, 'n': 1, 'ː': 1, 'd': 1, 'u': 1, ']': 1})
Counter({'θr': 1, 'ri': 1, 'is': 1, 'sv': 1, 'va': 1, 'ar': 1

Counter({'oː': 1, 'ːð': 1, 'ðs': 1, 'mɐ': 1, 'ɐy': 1, 'gɛ': 1, 'ɛv': 1, 'vn': 1, 'na': 1, 'a]': 1})
Counter({'ː': 4, 'θ': 2, 'r': 2, 'a': 2, '[': 1, 'o': 1, 'ɛ': 1, 'i': 1, 'n': 1, 'v': 1, ']': 1})
Counter({'θo': 1, 'oː': 1, 'ːr': 1, 'rː': 1, 'ɛi': 1, 'in': 1, 'nː': 1, 'θa': 1, 'ar': 1, 'va': 1, 'aː': 1, 'ː]': 1})
Counter({'n': 2, 'i': 2, 'ː': 2, '[': 1, 'θ': 1, 'r': 1, 'u': 1, 'ɣ': 1, 'm': 1, 'o': 1, 'ð': 1, ']': 1})
Counter({'θr': 1, 'ru': 1, 'un': 1, 'nɣ': 1, 'ɣi': 1, 'in': 1, 'nː': 1, 'mo': 1, 'oː': 1, 'ːð': 1, 'ði': 1, 'i]': 1})
Counter({'a': 3, 'n': 2, 's': 2, '[': 1, 'h': 1, 'ː': 1, 'j': 1, 'l': 1, 'd': 1, 'i': 1, 't': 1, 'r': 1, ']': 1})
Counter({'an': 2, 'ha': 1, 'nː': 1, 'sj': 1, 'ja': 1, 'al': 1, 'ld': 1, 'da': 1, 'si': 1, 'it': 1, 'tr': 1, 'r]': 1})
Counter({'ɛ': 2, 'r': 2, 'n': 2, 'ː': 2, '[': 1, 'h': 1, 'a': 1, 's': 1, 'l': 1, 'i': 1, 'k': 1, 't': 1, 'ɔ': 1, 'v': 1, 'f': 1, 'ɣ': 1, ']': 1})
Counter({'ɛr': 1, 'ha': 1, 'an': 1, 'nː': 1, 'sl': 1, 'li': 1, 'iː': 1, 'ːk': 1, '

...

Counter({'sa': 2, 'al': 1, 'aː': 1, 'hɔ': 1, 'ɔn': 1, 'st': 1, 'ta': 1, 'an': 1, 'nd': 1, 'da': 1, 'a]': 1})
Counter({'ː': 2, '[': 1, 's': 1, 'o': 1, 'l': 1, 'u': 1, 'f': 1, 'j': 1, 'a': 1, 'r': 1, 'i': 1, ']': 1})
Counter({'so': 1, 'oː': 1, 'ːl': 1, 'lu': 1, 'fj': 1, 'ja': 1, 'ar': 1, 'rː': 1, 'ːi': 1, 'i]': 1})
Counter({'n': 2, 'a': 2, 'ː': 2, '[': 1, 's': 1, 't': 1, 'r': 1, 'œ': 1, 'd': 1, 'u': 1, ']': 1})
Counter({'aː': 2, 'na': 1, 'ːs': 1, 'st': 1, 'tr': 1, 'rœ': 1, 'œn': 1, 'nd': 1, 'du': 1, 'ː]': 1})
Counter({'r': 4, 'ɔ': 2, '[': 1, 'n': 1, 'ð': 1, 'h': 1, 'v': 1, 'a': 1, 'd': 1, 'y': 1, 'ː': 1, ']': 1})
Counter({'ɔr': 2, 'nɔ': 1, 'rð': 1, 'ðr': 1, 'hɔ': 1, 'rv': 1, 'va': 1, 'dy': 1, 'yr': 1, 'rː': 1, 'ː]': 1})
Counter({'a': 3, 'r': 3, '[': 1, 'f': 1, 'l': 1, 'ː': 1, 'ɛ': 1, 'i': 1, 't': 1, 'd': 1, 'ɔ': 1, 'p': 1, ']': 1})
Counter({'fa': 1, 'al': 1, 'lː': 1, 'ːa': 1, 'ɛi': 1, 'it': 1, 'tr': 1, 'rd': 1, 'dr': 1, 'rɔ': 1, 'ɔp': 1, 'pa': 1, 'ar': 1, 'r]': 1})
Counter({'ː': 2, '[': 

Counter({'r': 2, '[': 1, 's': 1, 'o': 1, 'ː': 1, 't': 1, 'ɒ': 1, 'u': 1, 'ð': 1, 'h': 1, 'a': 1, 'n': 1, 'i': 1, ']': 1})
Counter({'so': 1, 'oː': 1, 'ːt': 1, 'tr': 1, 'rɒ': 1, 'ɒu': 1, 'uð': 1, 'ðr': 1, 'ha': 1, 'an': 1, 'ni': 1, 'i]': 1})
Counter({'a': 2, 'l': 2, '[': 1, 't': 1, 's': 1, 'œ': 1, 'u': 1, 'm': 1, 'h': 1, 'ɛ': 1, 'j': 1, 'r': 1, ']': 1})
Counter({'at': 1, 'sœ': 1, 'œl': 1, 'lu': 1, 'um': 1, 'hɛ': 1, 'ɛl': 1, 'lj': 1, 'ja': 1, 'ar': 1, 'r]': 1})
Counter({'r': 3, 'g': 2, 'm': 2, '[': 1, 'ɐ': 1, 'y': 1, 'n': 1, 'u': 1, 'ː': 1, 'a': 1, 'j': 1, 'œ': 1, 'k': 1, ']': 1})
Counter({'gɐ': 1, 'ɐy': 1, 'yr': 1, 'nu': 1, 'uː': 1, 'ga': 1, 'ar': 1, 'rm': 1, 'mr': 1, 'mj': 1, 'jœ': 1, 'œk': 1, 'k]': 1})
Counter({'i': 2, '[': 1, 'f': 1, 'y': 1, 'r': 1, 'g': 1, 'n': 1, 'p': 1, 'a': 1, 'h': 1, 'ɛ': 1, 'l': 1, 'ː': 1, ']': 1})
Counter({'fy': 1, 'yr': 1, 'gn': 1, 'ni': 1, 'ip': 1, 'pa': 1, 'ah': 1, 'hɛ': 1, 'ɛl': 1, 'lː': 1, 'ːi': 1, 'i]': 1})
Counter({'s': 2, 't': 2, 'n': 2, '[': 1, 'f': 1,

Counter({'a': 2, 'ː': 2, 'r': 2, '[': 1, 'θ': 1, 'k': 1, 'ɛ': 1, 'm': 1, 'h': 1, 'l': 1, 'i': 1, 'n': 1, ']': 1})
Counter({'θa': 1, 'aː': 1, 'kɛ': 1, 'ɛm': 1, 'mr': 1, 'hl': 1, 'li': 1, 'iː': 1, 'ːn': 1, 'na': 1, 'ar': 1, 'r]': 1})
Counter({'a': 4, 'r': 4, 'm': 2, 'ː': 2, '[': 1, 'h': 1, 'n': 1, 'f': 1, ']': 1})
Counter({'ar': 2, 'ha': 1, 'rm': 1, 'mr': 1, 'an': 1, 'nː': 1, 'ːa': 1, 'rː': 1, 'fr': 1, 'ra': 1, 'am': 1, 'm]': 1})
Counter({'ː': 3, 'ɛ': 2, 'r': 2, '[': 1, 'o': 1, 'ð': 1, 'i': 1, 'n': 1, 'f': 1, ']': 1})
Counter({'ɛr': 2, 'oː': 1, 'ːð': 1, 'ði': 1, 'in': 1, 'nː': 1, 'fɛ': 1, 'rː': 1, 'ː]': 1})
Counter({'v': 3, '[': 1, 'i': 1, 'ð': 1, 'u': 1, 'l': 1, 'ɛ': 1, 'ɣ': 1, 'a': 1, ']': 1})
Counter({'vi': 1, 'ið': 1, 'ul': 1, 'lv': 1, 'vɛ': 1, 'ɛɣ': 1, 'ɣa': 1, 'a]': 1})
Counter({'ɛ': 2, 'n': 2, 'b': 2, 'a': 2, '[': 1, 'i': 1, 'l': 1, 'j': 1, ']': 1})
Counter({'ɛn': 1, 'ba': 1, 'an': 1, 'ni': 1, 'bɛ': 1, 'ɛl': 1, 'lj': 1, 'ja': 1, 'a]': 1})
Counter({'r': 3, 't': 3, 'a': 2, '[': 1, '

Counter({'r': 3, 'g': 2, 'm': 2, '[': 1, 'ɐ': 1, 'y': 1, 'n': 1, 'u': 1, 'ː': 1, 'a': 1, 'j': 1, 'œ': 1, 'k': 1, ']': 1})
Counter({'gɐ': 1, 'ɐy': 1, 'yr': 1, 'nu': 1, 'uː': 1, 'ga': 1, 'ar': 1, 'rm': 1, 'mr': 1, 'mj': 1, 'jœ': 1, 'œk': 1, 'k]': 1})
Counter({'i': 2, '[': 1, 'f': 1, 'y': 1, 'r': 1, 'g': 1, 'n': 1, 'p': 1, 'a': 1, 'h': 1, 'ɛ': 1, 'l': 1, 'ː': 1, ']': 1})
Counter({'fy': 1, 'yr': 1, 'gn': 1, 'ni': 1, 'ip': 1, 'pa': 1, 'ah': 1, 'hɛ': 1, 'ɛl': 1, 'lː': 1, 'ːi': 1, 'i]': 1})
Counter({'s': 2, 't': 2, 'n': 2, '[': 1, 'f': 1, 'ɛ': 1, 'r': 1, 'm': 1, 'u': 1, 'l': 1, 'i': 1, 'a': 1, ']': 1})
Counter({'fɛ': 1, 'ɛs': 1, 'st': 1, 'tr': 1, 'mu': 1, 'un': 1, 'sl': 1, 'li': 1, 'it': 1, 'tn': 1, 'na': 1, 'a]': 1})
Counter({'ɛ': 3, 'n': 2, 'r': 2, '[': 1, 'f': 1, 'k': 1, 'i': 1, 'ː': 1, 'a': 1, ']': 1})
Counter({'ɛn': 2, 'rɛ': 2, 'fr': 1, 'ɛk': 1, 'ki': 1, 'nː': 1, 'ːa': 1, 'a]': 1})
Counter({'ɛ': 3, 'f': 2, 'ð': 2, '[': 1, 'j': 1, 'œ': 1, 'l': 1, 'v': 1, 'i': 1, 't': 1, 'k': 1, 'r': 1, 'ː

Counter({'so': 1, 'oː': 1, 'ːl': 1, 'lu': 1, 'fɛ': 1, 'ɛɣ': 1, 'ɣr': 1, 'ra': 1, 'a]': 1})
Counter({'a': 2, '[': 1, 'g': 1, 'u': 1, 'l': 1, 'ː': 1, 'i': 1, 'θ': 1, 'k': 1, 'ð': 1, 'n': 1, ']': 1})
Counter({'gu': 1, 'ul': 1, 'lː': 1, 'ːi': 1, 'θa': 1, 'ak': 1, 'kð': 1, 'ða': 1, 'an': 1, 'n]': 1})
Counter({'ː': 2, 'i': 2, '[': 1, 'a': 1, 'g': 1, 'm': 1, 'l': 1, 'e': 1, ']': 1})
Counter({'aː': 1, 'gi': 1, 'im': 1, 'ml': 1, 'le': 1, 'eː': 1, 'ːi': 1, 'i]': 1})
Counter({'a': 2, 'r': 2, 'u': 2, '[': 1, 'θ': 1, 's': 1, 'k': 1, 'l': 1, 'd': 1, 'y': 1, 'g': 1, 'ː': 1, 'v': 1, ']': 1})
Counter({'ar': 2, 'θa': 1, 'sk': 1, 'ku': 1, 'ul': 1, 'lu': 1, 'dy': 1, 'yg': 1, 'gː': 1, 'ːv': 1, 'va': 1, 'r]': 1})
Counter({'ː': 3, 'r': 2, '[': 1, 'd': 1, 'o': 1, 't': 1, 'i': 1, 'b': 1, 'y': 1, 'g': 1, 'j': 1, 'a': 1, ']': 1})
Counter({'dr': 1, 'ro': 1, 'oː': 1, 'ːt': 1, 'tː': 1, 'ːi': 1, 'ir': 1, 'by': 1, 'yg': 1, 'gː': 1, 'ːj': 1, 'ja': 1, 'a]': 1})
Counter({'a': 3, 'd': 2, '[': 1, 'ɔ': 1, 'k': 1, 'u': 1, '

#### show vocabulary

In [84]:
from cltk.utils.frequency import Frequency

frq = Frequency()

text = " ".join([verse for paragraph in paragraphs for verse in paragraph])
times = frq.counter_from_str(text)
times.most_common(50)

[('ok', 49),
 ('er', 25),
 ('á', 24),
 ('of', 22),
 ('í', 20),
 ('en', 18),
 ('hon', 18),
 ('ek', 14),
 ('um', 14),
 ('at', 12),
 ('var', 11),
 ('né', 11),
 ('enn', 11),
 ('eða', 11),
 ('sá', 11),
 ('þat', 10),
 ('hvat', 10),
 ('Þá', 10),
 ('fyr', 9),
 ('ór', 9),
 ('mun', 9),
 ('Vituð', 9),
 ('ér', 9),
 ('þar', 8),
 ('öll', 7),
 ('af', 7),
 ('mjök', 6),
 ('veit', 6),
 ('þá', 5),
 ('þeir', 5),
 ('hann', 5),
 ('æsir', 5),
 ('þeim', 5),
 ('nú', 5),
 ('til', 5),
 ('Óðinn', 5),
 ('koma', 5),
 ('inn', 5),
 ('ragna', 5),
 ('fram', 5),
 ('munu', 5),
 ('man', 4),
 ('gengu', 4),
 ('regin', 4),
 ('rökstóla', 4),
 ('ginnheilög', 4),
 ('goð', 4),
 ('gættusk', 4),
 ('dverga', 4),
 ('annarr', 4)]

#### show syntax